In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import math
from torch.distributions.exponential import Exponential
from torch.distributions.beta import Beta
from torch.distributions.pareto import Pareto

def RMSE(results, true):
    # results: tensor with 2 dimensions
    # array or list
    sim = results.size()[0]
    p = results.size()[1]
    
    tmp = 0
    for i in range(p):
        tmp += torch.sum(torch.square(results[:,i] - true[i] ))/sim
    
    return torch.sqrt(tmp).detach().cpu().numpy().tolist()

def OU_simul_sample(L_OU, time_OU, y0_OU, mu_OU, theta_OU, sigma_OU):
    """
    This function generates a sample path for OU process
    L_OU: number of simulation number
    y0: initial value
    t: time array including initial point
    mu, theta, sigma are parameters of OU process dX_t = theta (mu - X_t)dt + sigma dBt
    """
    z0 = y0_OU
    path_OU = torch.zeros(L_OU, time_OU.size)
    path_OU[:,0] = y0_OU
    for l in range(time_OU.size-1):
        del_L = time_OU[l+1] - time_OU[l]
        OU_mean = z0 * torch.exp(-mu_OU * del_L) + theta_OU * (1- torch.exp(-mu_OU * del_L))
        OU_sd = torch.sqrt( sigma_OU**2/(2*mu_OU) * (1- torch.exp(-2 * mu_OU * del_L)) )
        z0 = torch.normal(OU_mean, OU_sd)
        path_OU[:,l+1] = z0
    return(path_OU)

def CIR_simul_sample(obtime, a, b, sigma, y0 = None):
    """
    This function generates a one sample path between an interval for CIR process
    y0_CIR: initial value
    m : num of slice of each interval
    t_CIR: time array including initial point
    a, b, sigma are parameters of CIR process dX_t = a (b - X_t)dt + \sigma sqrt{X_t} dBt
    """
    if y0 is None:
        shape = 2 * a * b
        shape = shape.numpy()
        scale = sigma ** 2 / (2 * a)
        scale = scale.numpy()
        y0 = np.random.gamma(shape, scale)
        y0 = torch.from_numpy(y0)

    L = a.size()[0]
    z0 = y0
    
    path = torch.zeros(L, obtime.size)
    path[:,0] = y0
    
    nu0 = 4 * a * b / sigma ** 2
    nu0 = nu0.numpy()
    
    for l in range(obtime.size-1):
        del_L = obtime[l+1] - obtime[l]
        c0 = 4 * a / sigma ** 2 / (1- torch.exp(-a * del_L))
        lambda0 = c0 * z0 * torch.exp(-a * del_L)
        lambda0 = lambda0.numpy()
        tmp = np.random.noncentral_chisquare(nu0, lambda0)
        tmp = torch.from_numpy(tmp)
        z0 = tmp/c0
        path[:,l+1] = z0
    return(path)

def Jacobi_sample(del_Jacobi, m_Jacobi, y0_Jacobi, a_Jacobi, b_Jacobi, sigma_Jacobi):
    """
    This function generates a one sample path between an interval for Jacobi process
    y0_Jacobi: initial value
    m : num of slice of each interval
    t_Jacobi: time array including initial point
    a, b, sigma are parameters of Jacobi process dX_t = a (b - X_t)dt + \sigma sqrt{X_t (1- X_t)} dBt
    """
    L_tmp = a_Jacobi.size()[0]
    del_tmp = del_Jacobi / m_Jacobi
    z0 = y0_Jacobi
    for j in range(m_Jacobi):
        ran_num = torch.normal(0 * torch.ones(L_tmp),1 * torch.ones(L_tmp))
        z0 = z0 + a_Jacobi*(b_Jacobi-z0)*del_tmp + sigma_Jacobi*torch.mul(torch.sqrt(z0),torch.sqrt(torch.ones(L_tmp)-z0))*((del_tmp)**(1/2))*ran_num + (torch.pow(sigma_Jacobi,2) )/4*(torch.ones(L_tmp)-2*z0)*(del_tmp*(torch.pow(ran_num,2)-1)) # Milstein approximation
        z0 = torch.max(z0,0 * torch.ones(L_tmp))
        z0 = torch.min(z0,1 * torch.ones(L_tmp))
    return(z0)

def Jacobi_simul_sample(L_Jacobi, time_Jacobi, m_Jacobi, y0_Jacobi, a_Jacobi, b_Jacobi, sigma_Jacobi):
    """
    This function generates a sample path for Jacobi process
    L_Jacobi: number of simulation number
    y0_Jacobi: initial value
    m : num of slice of each interval
    t_Jacobi: time array including initial point
    a, b, sigma are parameters of Jacobi process dX_t = a (b - X_t)dt + \sigma sqrt{X_t (1- X_t)} dBt
    """
    tmp = y0_Jacobi
    path_Jacobi = torch.zeros(L_Jacobi, time_Jacobi.size)
    path_Jacobi[:,0] = y0_Jacobi
    for l in range(time_Jacobi.size-1):
        del_L = time_Jacobi[l+1] - time_Jacobi[l]
        new_path = Jacobi_sample(del_L, m_Jacobi, tmp, a_Jacobi, b_Jacobi, sigma_Jacobi) 
        path_Jacobi[:,l+1] = new_path
        tmp = torch.clone(new_path)
    return(path_Jacobi)

def TriOU_simul_sample(x01_TriOU, x02_TriOU, x03_TriOU, t_TriOU, m0, kappa_11, kappa_21, kappa_22, kappa_31, kappa_32, kappa_33):
    """
    This function generates a sample path for Triple OU process
    """
    
    x01 = x01_TriOU
    x02 = x02_TriOU
    x03 = x03_TriOU

    L_TriOU = x01_TriOU.size()[0]

    path_TriOU = torch.zeros(L_TriOU, 3, t_TriOU.size)
    
    path_TriOU[:,0,0] = x01_TriOU
    path_TriOU[:,1,0] = x02_TriOU
    path_TriOU[:,2,0] = x03_TriOU
    
    for l in range(t_TriOU.size-1):
        # X, Y generating
        del_x = t_TriOU[l+1] - t_TriOU[l]
        del_del = del_x / m0

        for j in range(m0):
            ran_num1 = torch.normal(0 * torch.ones(L_TriOU), 1 * torch.ones(L_TriOU))
            ran_num2 = torch.normal(0 * torch.ones(L_TriOU), 1 * torch.ones(L_TriOU))
            ran_num3 = torch.normal(0 * torch.ones(L_TriOU), 1 * torch.ones(L_TriOU))
            
            x01 =  x01 - kappa_11*  x01 * del_del + ((del_del)**(1/2)) * ran_num1
            x02 =  x02 + ( - kappa_21*  x01 - kappa_22 * x02)  * del_del + ((del_del)**(1/2)) * ran_num2   # Euler approximation
            x03 =  x03 + ( - kappa_31*  x01 - kappa_32 * x02 - kappa_33 * x03 )  * del_del + ((del_del)**(1/2)) * ran_num3   # Euler approximation

        path_TriOU[:,0,l+1] = x01
        path_TriOU[:,1,l+1] = x02
        path_TriOU[:,2,l+1] = x03
    return(path_TriOU)

def MROUJ_simul_sample(obtime, m, y0, kappa, beta, sigma, lamb, mu):
    """
    This function generates a one sample path between an interval for VNJ process
    dX_t = \kappa(beta-X_t)dt + sigma dB_t + J_t where N_t: Poisson process with lamb, Z_i ~ Exp(mu)
    y0_VNJ: initial value
    m : num of slice of each interval
    """
    L_tmp = kappa.size()[0]
    z0 = y0
    path = torch.zeros(L_tmp, obtime.size)
    path[:,0] = z0

    for l in range(len(obtime)-1):
        # X, Y generating
        del_x = obtime[l+1] - obtime[l]
        del_y = del_x / m

        for j in range(m):
            ran_num = torch.normal(0 * torch.ones(L_tmp), torch.ones(L_tmp))
            ran_num2 = Exponential(mu * torch.ones(L_tmp)).sample() # rate
            ran_num3 = torch.poisson(torch.ones(L_tmp) * lamb * del_y)
            z0 = z0 + kappa*(beta-z0)*del_y + sigma * ran_num * del_y ** (1/2) + ran_num2 * ran_num3
        path[:,l+1] = z0
    return(path)


def SQRJ_simul_sample(obtime, m, y0, kappa, beta, sigma, lamb, mu):
    """
    This function generates a one sample path between an interval for VNJ process
    dX_t = \kappa(beta-X_t)dt + sigma dB_t + J_t where N_t: Poisson process with lamb, Z_i ~ Exp(mu)
    y0_VNJ: initial value
    m : num of slice of each interval
    """
    L_tmp = kappa.size()[0]
    z0 = y0
    path = torch.zeros(L_tmp, obtime.size)
    path[:,0] = z0

    for l in range(len(obtime)-1):
        # X, Y generating
        del_x = obtime[l+1] - obtime[l]
        del_y = del_x / m

        for j in range(m):
            ran_num = torch.normal(0 * torch.ones(L_tmp), torch.ones(L_tmp))
            ran_num2 = Exponential(mu * torch.ones(L_tmp)).sample() # rate
            ran_num3 = torch.poisson(torch.ones(L_tmp) * lamb * del_y)
            z0 = z0 + kappa*(beta-z0)*del_y + sigma * torch.sqrt(z0) * ran_num * del_y ** (1/2) + sigma ** 2 /2 *(del_y*(torch.pow(ran_num,2)-1)) + ran_num2 * ran_num3 # Milstein + Jump
            z0 = torch.max(z0,1e-10 * torch.ones(L_tmp))
        path[:,l+1] = z0
    return(path)



def BOUJ_simul_sample(obtime, m, x01, x02, kappa_11, kappa_21, kappa_22, theta_1, theta_2, sigma_1, sigma_2, lamb_1, lamb_2, mu_1, mu_2):
    """
    This function generates a sample path for bivariate OU process
    """
    L = kappa_11.size()[0]
    z01 = x01
    z02 = x02
    
    path = torch.zeros(2, L, len(obtime))
    path[0,:,0] = z01
    path[1,:,0] = z02

    for l in range(len(obtime)-1):
        # X, Y generating
        del_x = obtime[l+1] - obtime[l]
        del_y = del_x / m

        for j in range(m):
            ran_num1 = torch.normal(0 * torch.ones(L), 1 * torch.ones(L))
            ran_num2 = torch.normal(0 * torch.ones(L), 1 * torch.ones(L))
            
            ran_num3 = torch.poisson( torch.ones(L) * lamb_1 * del_y)
            ran_num4 = torch.poisson( torch.ones(L) * lamb_2 * del_y)
            
            ran_num5 = Exponential(mu_1 * torch.ones(L)).sample() # rate
            ran_num6 = Exponential(mu_2 * torch.ones(L)).sample() # rate
            
            z01 = z01 + kappa_11* (theta_1 - z01 ) * del_y 
            z01 = z01 + (del_y**(1/2)) * sigma_1 * ran_num1
            z01 = z01 + ran_num3 * ran_num5
            
            z02 = z02 + (kappa_21* (theta_1-z01)+kappa_22*(theta_2-z02))*del_y 
            z02 = z02 + (del_y**(1/2)) * sigma_2* ran_num2  
            z02 = z02 + ran_num4 * ran_num6
            
        path[0,:,l+1] = z01
        path[1,:,l+1] = z02
    return(path)


def PBJD_simul_sample(obtime, y0, beta, sigma, lamb_p, lamb_n, eta_p, eta_n):
    """
    This function generates a one sample path between an interval for PBJD process
    dX_t = muX_tdt + sigma dB_t + J_1t + J_2t 
    m : num of slice of each interval
    """
    L_tmp = beta.size()[0]
    z0 = y0
    path = torch.zeros(L_tmp, obtime.size)
    path[:,0] = z0

    for l in range(len(obtime)-1):
        # X, Y generating
        del_x = obtime[l+1] - obtime[l]

        ran_num = torch.normal(0 * torch.ones(L_tmp), torch.ones(L_tmp))
            
        # jump to positive
        ran_num2 = torch.zeros(L_tmp)
        tmp = torch.poisson( torch.ones(L_tmp) * lamb_p * del_x)
        for i in range(int(torch.max(tmp))+1):
            if (i > 0) and (tmp == i).sum() > 0:
                eta_tmp = eta_p[tmp ==i]
                eta_tmp = eta_tmp.repeat(i, 1)
                tmp3 = torch.log(Pareto(torch.tensor([1.0]), eta_tmp).sample())
                tmp3 = torch.sum(tmp3, 0)
                ran_num2[tmp == i] = tmp3
        ran_num2 = torch.min(ran_num2, torch.ones(L_tmp) * 1000)
            
        # jump to negative
        ran_num3 = torch.zeros(L_tmp)
        tmp = torch.poisson( torch.ones(L_tmp) * lamb_n * del_x)
        for i in range(int(torch.max(tmp))+1):
            if (i > 0) and (tmp == i).sum() > 0:
                eta_tmp = eta_n[tmp ==i]
                eta_tmp = eta_tmp.repeat(i, 1)
                tmp3 = torch.log(Beta(eta_tmp, torch.tensor([1.0])).sample())
                tmp3 = torch.sum(tmp3, 0)
                ran_num3[tmp == i] = tmp3
        ran_num3 = torch.max(ran_num3, -torch.ones(L_tmp) * 1000)
        
        z0 = z0 + (beta - sigma ** 2/ 2) * del_x + sigma * ran_num * del_x ** (1/2) + ran_num2 + ran_num3
        #z0 = z0 + (beta) * del_x + sigma * ran_num * del_x ** (1/2) + ran_num2 + ran_num3
        
        path[:,l+1] = z0
    return(path)

In [2]:
def MLE_OU(sim_OU, obtime_OU, delta_OU):
    """
    sim_OU size : sim * n_OU
    """
    sim_num = sim_OU.size()[0]
    n_OU = len(obtime_OU)
    n0_OU = n_OU -1
    vec_xi = sim_OU[:, 1:]
    vec_xim1 = sim_OU[:, :n0_OU]
    beta1 = (torch.sum(vec_xi * vec_xim1,1) - 1/n0_OU * torch.sum(vec_xi, 1) * torch.sum(vec_xim1, 1) ) / (torch.sum(vec_xim1 ** 2, 1) - 1/n0_OU * (torch.sum(vec_xim1, 1))**2   ) # size : sim
    beta2 = 1/n0_OU * (  torch.sum(vec_xi,1) - beta1 * torch.sum(vec_xim1,1) ) / (torch.ones(sim_num)-beta1) # size: sim

    tmp1 = beta1.repeat(n_OU-1, 1) # sim *n_OU
    tmp1 = torch.transpose(tmp1, 0,1)
    tmp2 = beta2 * (torch.ones(sim_num)-beta1)
    tmp3 = tmp2.repeat(n_OU-1, 1)
    tmp3 = torch.transpose(tmp3, 0,1)
    tmp_beta3 = (vec_xi - tmp1 * vec_xim1 - tmp3) # size: sim * n_OU
    beta3 = 1/n0_OU * torch.sum(tmp_beta3**2, 1)
    mu = -torch.ones(sim_num) * torch.log(beta1) / delta_OU
    alpha = beta2
    sigma = torch.sqrt(2 * mu * beta3 / (torch.ones(sim_num) - beta1 ** 2 ))

    return(torch.stack((mu, alpha,sigma ** 2), 1))

def PMLE_CIR(sim_CIR, obtime_CIR, delta_CIR):
    sim_num = sim_CIR.size()[0]
    n_CIR = len(obtime_CIR)
    n0_CIR = n_CIR -1
    vec_xi = sim_CIR[:, 1:]
    vec_xim1 = sim_CIR[:, :n0_CIR]
    beta1 = (- torch.sum(vec_xi / vec_xim1,1) + 1/n0_CIR * torch.sum(vec_xi, 1) * torch.sum(1/vec_xim1, 1) ) / (1/n0_CIR * torch.sum(vec_xim1, 1) * torch.sum((1/vec_xim1), 1) - n0_CIR  ) # size : sim
    beta2 = (1/n0_CIR * (  torch.sum(vec_xi / vec_xim1, 1)) - beta1 ) / ((torch.ones(sim_num)-beta1) * 1/n0_CIR * torch.sum(1/vec_xim1, 1) ) # size: sim
    
    tmp1 = beta1.repeat(n_CIR-1, 1) # sim *n_CIR
    tmp1 = torch.transpose(tmp1, 0,1)
    tmp2 = beta2 * (torch.ones(sim_num)-beta1)
    tmp3 = tmp2.repeat(n_CIR-1, 1)
    tmp3 = torch.transpose(tmp3, 0,1)
    tmp_beta3 = (vec_xi - tmp1 * vec_xim1 - tmp3) # size: sim * n_CIR
    beta3 = 1/n0_CIR * torch.sum(tmp_beta3**2 /vec_xim1, 1)
    
    mu = -torch.ones(sim_num) * torch.log(beta1) / delta_CIR
    alpha = beta2
    sigma = torch.sqrt(2 * mu * beta3 / (torch.ones(sim_num) - beta1 ** 2 ))

    return(torch.stack((mu, alpha,sigma ** 2), 1))

def LSE_TriOU(sim_data, delta_ob):
    A = torch.matmul(sim_data[:,:,1:(n+1)], torch.transpose(sim_data[:,:,0:n],1,2 ))
    B = torch.linalg.inv(torch.matmul(sim_data[:,:,0:n],torch.transpose(sim_data[:,:,0:n],1,2) ))
    Fhat = torch.matmul(A,B)
    diag3 = torch.diag(torch.tensor([1,1,1]))
    Ahat1 = -(Fhat - diag3) / delta_ob
    Ahat2 = -2* torch.matmul((Fhat - diag3), torch.linalg.inv(Fhat + diag3)) / delta_ob

    Ahat1 = torch.stack((Ahat1[:,0,0], Ahat1[:,1,0], Ahat1[:,1,1], Ahat1[:,2,0], Ahat1[:,2,1], Ahat1[:,2,2]), 1)
    Ahat2 = torch.stack((Ahat2[:,0,0], Ahat2[:,1,0], Ahat2[:,1,1], Ahat2[:,2,0], Ahat2[:,2,1], Ahat2[:,2,2]), 1)

    return([Ahat1,Ahat2])

In [1]:
def OU_summary(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]

    sum1 = torch.zeros(L0) # sum x_i x_{i-1}
    sum2 = torch.zeros(L0) # sum x_i
    sum3 = torch.zeros(L0) # sum x_{i-1}
    sum4 = torch.zeros(L0) # sum x_{i-1}^2
    sum5 = torch.zeros(L0) # sum x_{i}^2

    for l in range(n0-1):
      sum1 = sum1 + X[:,l+1] * X[:,l]
      sum2 = sum2 + X[:,l+1]
      sum3 = sum3 + X[:,l]
      sum4 = sum4 + torch.pow(X[:,l],2)
      sum5 = sum5 + torch.pow(X[:,l+1],2)
    return(torch.stack(( (sum1 - sum2 * sum3/n0 ) /n0, sum2/n0, sum3/n0, sum4/n0 - (sum3/n0)**2, sum5/n0 - (sum2/n0)**2) ,1))

def OU_summary2(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]

    L0 = X.size()[0]
    n0 = X.size()[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    
    s2 = torch.mean((X0 - s0) * (X1 - s1),1,keepdim=True)
    s3 = torch.mean((X0 - s0)**2,1, keepdim=True)
    s4 = torch.mean((X1 - s1)**2,1, keepdim=True)
    
    s5 = torch.mean((X0 - s0)**2 * (X1 - s1), 1, keepdim=True) /n0
    s6 = torch.mean((X0 - s0) * (X1 - s1)**2, 1, keepdim=True) /n0
    s7 = torch.mean((X0 - s0)**2 * (X1 - s1)**2, 1, keepdim=True) / (n0 ** 2)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7)) ) 


def CIR_summary(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    
    s2 = torch.mean((X0 - s0) * (X1 - s1),1,keepdim=True)
    s3 = torch.mean((X0 - s0)**2,1, keepdim=True)
    s4 = torch.mean((X1 - s1)**2,1, keepdim=True)
    
    s5 = torch.mean(1/ X0, 1, keepdim=True)
    s6 = torch.mean(X1/ X0, 1, keepdim=True)
    s7 = torch.mean(X1 ** 2/ X0, 1, keepdim=True)
    s8 = torch.log(torch.max(X0, 1e-10*torch.ones(1)))
    s8 = torch.mean(s8, 1, keepdim=True)
    
    s9 = torch.mean((X0 - s0)**2 * (X1 - s1), 1, keepdim=True)
    s10 = torch.mean((X0 - s0) * (X1 - s1)**2, 1, keepdim=True)
    s11 = torch.mean((X0 - s0)**2 * (X1 - s1)**2, 1, keepdim=True)
    
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11)) ) 

def CIR_summary2(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    
    s2 = torch.mean((X0 - s0) * (X1 - s1),1,keepdim=True)
    s3 = torch.mean((X0 - s0)**2,1, keepdim=True)
    s4 = torch.mean((X1 - s1)**2,1, keepdim=True)
    
    s5 = torch.mean(1/ X0, 1, keepdim=True)
    s6 = torch.mean(X1/ X0, 1, keepdim=True)
    s7 = torch.mean(X1 ** 2/ X0, 1, keepdim=True)
    
    s8 = torch.mean((X0 - s0)**2 * (X1 - s1), 1, keepdim=True)
    s9 = torch.mean((X0 - s0) * (X1 - s1)**2, 1, keepdim=True)
    s10 = torch.mean((X0 - s0)**2 * (X1 - s1)**2, 1, keepdim=True)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10)) ) 


def CIR_summary3(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    
    s2 = torch.mean((X0 - s0) * (X1 - s1), 1, keepdim=True)
    s3 = torch.mean((X0 - s0)**2, 1, keepdim=True)
    s4 = torch.mean((X1 - s1)**2, 1, keepdim=True)
    
    s5 = torch.mean(1/ X0, 1, keepdim=True)
    s6 = torch.mean(X1/ X0, 1, keepdim=True)
    s7 = torch.mean(X1 ** 2/ X0, 1, keepdim=True)
    
    s8 = torch.log(torch.max(X0, 1e-10*torch.ones(1)))
    s8 = torch.mean(s8, 1, keepdim=True)
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8)) ) 

def Jacobi_summary(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    X1 = X[:,:-1]
    X2 = X[:,1:]
    
    s1 = torch.mean(X[:,:-1], 1, keepdim=True) # mean of x_{i-1}
    s2 = torch.mean(X[:,1:], 1, keepdim=True) # mean of x_i
    
    s3 = torch.mean((X1 - s1) * (X2 - s2),1,keepdim=True)
    s4 = torch.mean((X1 - s1)**2,1, keepdim=True)
    s5 = torch.mean((X2 - s2)**2,1, keepdim=True)
    
    s6 = torch.mean((X1 - s1)**2 * (X2 - s2), 1, keepdim=True)
    s7 = torch.mean((X1 - s1) * (X2 - s2)**2, 1, keepdim=True)
    
    s8 = torch.mean((X1 - s1)**2 * (X2 - s2)**2, 1, keepdim=True)
      
    return(torch.column_stack((s1, s2, s3, s4, s5, s6, s7, s8)) ) 

def TriOU_summary(ten):
    """
    X: torch size: [L,n]
    """
    L0 = ten.size()[0]
    n0 = ten.size()[2]

    sum1_1 = torch.zeros(L0) # sum x_{i-1}
    sum1_2 = torch.zeros(L0) # sum x_{i-1}^2
    sum1_3 = torch.zeros(L0) # sum x_{i}^2
    sum1_4 = torch.zeros(L0) # sum x_{i} * x_{i-1}
    sum1_5 = torch.zeros(L0) # sum y_{i-1}
    
    sum2_1 = torch.zeros(L0) # sum y_{i-1}^2
    sum2_2 = torch.zeros(L0) # sum y_{i}
    sum2_3 = torch.zeros(L0) # sum y_{i}^2
    sum2_4 = torch.zeros(L0) # sum y_{i} * y_{i-1}
    sum2_5 = torch.zeros(L0) # sum y_{i} * y_{i-1}
    
    sum3_1 = torch.zeros(L0) # sum z_{i-1}^2
    sum3_2 = torch.zeros(L0) # sum z_{i}
    sum3_3 = torch.zeros(L0) # sum z_{i}^2
    sum3_4 = torch.zeros(L0) # sum z_{i} * z_{i-1}
    sum3_5 = torch.zeros(L0) # sum z_{i} * z_{i-1}
    
    sum12_1 = torch.zeros(L0) # sum_x_{i-1} y_{i-1}
    sum12_2 = torch.zeros(L0) # sum_x_{i} y_{i-1}
    sum12_3 = torch.zeros(L0) # sum x_{i-1} y_i
    sum12_4 = torch.zeros(L0) # sum x_{i} y_{i}
    sum12_5 = torch.zeros(L0) # sum x_{i}^2 y_{i}^2
    sum12_6 = torch.zeros(L0) # sum x_{i-1}^2 y_{i-1}^2
    
    sum23_1 = torch.zeros(L0) # sum_y_{i-1} z_{i-1}
    sum23_2 = torch.zeros(L0) # sum_y_{i} z_{i-1}
    sum23_3 = torch.zeros(L0) # sum y_{i-1} z_i
    sum23_4 = torch.zeros(L0) # sum y_{i} z_{i}
    sum23_5 = torch.zeros(L0) # sum y_{i}^2 z_{i}^2
    sum23_6 = torch.zeros(L0) # sum y_{i-1}^2 z_{i-1}^2
    
    sum13_1 = torch.zeros(L0) # sum_x_{i-1} z_{i-1}
    sum13_2 = torch.zeros(L0) # sum_x_{i} z_{i-1}
    sum13_3 = torch.zeros(L0) # sum x_{i-1} z_i
    sum13_4 = torch.zeros(L0) # sum x_{i} z_{i}
    sum13_5 = torch.zeros(L0) # sum x_{i}^2 z_{i}^2
    sum13_6 = torch.zeros(L0) # sum x_{i-1}^2 z_{i-1}^2
    
    sum123_1 = torch.zeros(L0) # 
    sum123_2 = torch.zeros(L0) # 
    sum123_3 = torch.zeros(L0) # 
    sum123_4 = torch.zeros(L0) # 
    sum123_5 = torch.zeros(L0) # 
    sum123_6 = torch.zeros(L0) # 
    sum123_7 = torch.zeros(L0) # 

    for l in range(n0-1):
      sum1_1 = sum1_1 + ten[:,0,l]
      sum1_2 = sum1_2 + torch.pow(ten[:,0,l], 2)
      sum1_3 = sum1_3 + ten[:,0,(l+1)]
      sum1_4 = sum1_4 + torch.pow(ten[:,0,l+1], 2)
      sum1_5 = sum1_5 + ten[:,0,l] * ten[:,0,l+1]
      
      sum2_1 = sum2_1 + ten[:,1,l]
      sum2_2 = sum2_2 + torch.pow(ten[:,1,l], 2)
      sum2_3 = sum2_3 + ten[:,1,(l+1)]
      sum2_4 = sum2_4 + torch.pow(ten[:,1,l+1], 2)
      sum2_5 = sum2_5 + ten[:,1,l] * ten[:,1,l+1]
      
      sum3_1 = sum2_1 + ten[:,2,l]
      sum3_2 = sum2_2 + torch.pow(ten[:,2,l], 2)
      sum3_3 = sum2_3 + ten[:,2,(l+1)]
      sum3_4 = sum2_4 + torch.pow(ten[:,2,l+1], 2)
      sum3_5 = sum2_5 + ten[:,2,l] * ten[:,2,l+1]
      
      sum12_1 = sum12_1 + ten[:,0,l] * ten[:,1,l]
      sum12_2 = sum12_2 + ten[:,0,l+1] * ten[:,1,l]
      sum12_3 = sum12_3 + ten[:,0,l] * ten[:,1,l+1]
      sum12_4 = sum12_4 + ten[:,0,l+1] * ten[:,1,l+1]
      sum12_5 = sum12_5 + torch.pow(ten[:,0,l+1],2) * ten[:,1,l+1]
      sum12_6 = sum12_6 + torch.pow(ten[:,0,l+1],2) * torch.pow(ten[:,1,l+1],2)
      
      sum23_1 = sum23_1 + ten[:,1,l] * ten[:,2,l]
      sum23_2 = sum23_2 + ten[:,1,l+1] * ten[:,2,l]
      sum23_3 = sum23_3 + ten[:,1,l] * ten[:,2,l+1]
      sum23_4 = sum23_4 + ten[:,1,l+1] * ten[:,2,l+1]
      sum23_5 = sum23_5 + torch.pow(ten[:,1,l+1],2) * ten[:,2,l+1]
      sum23_6 = sum23_6 + torch.pow(ten[:,1,l+1],2) * torch.pow(ten[:,2,l+1],2)
      
      sum13_1 = sum13_1 + ten[:,0,l] * ten[:,2,l]
      sum13_2 = sum13_2 + ten[:,0,l+1] * ten[:,2,l]
      sum13_3 = sum13_3 + ten[:,0,l] * ten[:,2,l+1]
      sum13_4 = sum13_4 + ten[:,0,l+1] * ten[:,2,l+1]
      sum13_5 = sum13_5 + torch.pow(ten[:,0,l+1],2) * ten[:,2,l+1]
      sum13_6 = sum13_6 + torch.pow(ten[:,0,l+1],2) * torch.pow(ten[:,2,l+1],2)

      sum123_1 = sum123_1 + ten[:,0,l+1] * ten[:,1,l+1] * ten[:,2,l+1]
      sum123_2 = sum123_2 + ten[:,0,l] * ten[:,1,l+1] * ten[:,2,l+1]
      sum123_3 = sum123_3 + ten[:,0,l+1] * ten[:,1,l] * ten[:,2,l+1]
      sum123_4 = sum123_4 + ten[:,0,l+1] * ten[:,1,l+1] * ten[:,2,l]
      
      sum123_5 = sum123_5 + torch.pow(ten[:,0,l+1],2) * ten[:,1,l+1] * ten[:,2,l+1]
      sum123_6 = sum123_6 + ten[:,0,l+1] * torch.pow(ten[:,1,l+1],2) * ten[:,2,l+1]
      sum123_7 = sum123_7 + ten[:,0,l+1] * ten[:,1,l+1] * torch.pow(ten[:,2,l+1],2)
      
    stack_x = torch.stack((sum1_1/n0, sum1_2/n0, sum1_3/n0, sum1_4/n0, sum1_5/n0),1) #5
    stack_y = torch.stack((sum2_1/n0, sum2_2/n0, sum2_3/n0, sum2_4/n0, sum2_5/n0),1) #5
    stack_z = torch.stack((sum3_1/n0, sum3_2/n0, sum3_3/n0, sum3_4/n0, sum3_5/n0),1) #5
    
    stack_xy = torch.stack((sum12_1/n0, sum12_2/n0, sum12_3/n0, sum12_4/n0, sum12_5/n0, sum12_6/n0), 1) #6
    stack_yz = torch.stack((sum23_1/n0, sum23_2/n0, sum23_3/n0, sum23_4/n0, sum23_5/n0, sum23_6/n0), 1) #6
    stack_zx = torch.stack((sum13_1/n0, sum13_2/n0, sum13_3/n0, sum13_4/n0, sum13_5/n0, sum13_6/n0), 1) #6
    
    stack_xyz = torch.stack((sum123_1/n0, sum123_2/n0, sum123_3/n0, sum123_4/n0, sum123_5/n0, sum123_6/n0, sum123_7/n0 ), 1) # 7
     
    return(torch.cat([stack_x, stack_y, stack_z,stack_xy, stack_yz, stack_zx, stack_xyz], 1))

from torch.distributions.exponential import Exponential
def MROUJ_summary(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    Xi = X[:,range(1,n0)]
    Xi1 = X[:,range(0,n0-1)]
    
    s0 = torch.mean(Xi, 1)
    s1 = torch.mean(Xi1, 1)
     
    Xi = Xi - torch.reshape(s0, (L0, 1))
    Xi1 = Xi1 - torch.reshape(s1, (L0, 1))
    
    s2 = torch.mean(Xi * Xi1, 1) / n0
    s3 = torch.mean(Xi **2 , 1) /n0
    s4 = torch.mean(Xi1 **2 , 1) /n0
    
    s5 = torch.mean(torch.abs(Xi - Xi1), 1)
    s6 = torch.mean((Xi - Xi1)**2 , 1) / n0 
    s7 = torch.mean((Xi - Xi1)**3 , 1) / n0
    s8 = torch.mean((Xi - Xi1)**4 , 1) / n0 ** 2
    
    s9 = torch.mean((Xi - Xi1)**2 * Xi, 1) / n0 
    s10 = torch.mean((Xi - Xi1)**2 * Xi ** 2, 1)/ n0 ** 2
    
    Xi = Xi + torch.reshape(s0, (L0, 1))
    Xi1 = Xi1 + torch.reshape(s1, (L0, 1))
    
    s11 = torch.mean(Xi * Xi1, 1) - s0 * s1
    s11 = s11 / ( torch.mean(Xi1 ** 2, 1) - s1 ** 2 )
    
    s12 = s0 * torch.mean(Xi1 ** 2,1) - s1 * torch.mean(Xi * Xi1, 1)
    s12 = s12 / (torch.mean(Xi1 ** 2, 1) - s1 ** 2)
    
    # Jump intensity
    tmp = abs(Xi - Xi1)
    
    thres = [1e-5 * 3, 1e-5 * 6, 1e-5 * 9, 1e-4 * 3, 1e-4 * 6, 1e-4 * 9, 1e-3 * 3, 1e-3 * 6, 1e-3 * 9,
           1e-2 * 3, 1e-2 * 6, 1e-2 * 9, 1e-1 * 3, 1e-1 * 6, 1e-1 * 9,
             1.0, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)

    j_int = torch.column_stack(thres_tmp)
    
    # Jump magnitude
    tmp = Xi - Xi1
    num = 33
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, 
                               s10, s11, s12, j_int, mag_q)) ) 

def MROUJ_summary2(X):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    Xi = X[:,range(1,n0)]
    Xi1 = X[:,range(0,n0-1)]
    
    s0 = torch.mean(Xi, 1)
    s1 = torch.mean(Xi1, 1)
     
    Xi = Xi - torch.reshape(s0, (L0, 1))
    Xi1 = Xi1 - torch.reshape(s1, (L0, 1))
    
    s2 = torch.mean(Xi * Xi1, 1) / n0
    s3 = torch.mean(Xi **2 , 1) /n0
    s4 = torch.mean(Xi1 **2 , 1) /n0
    
    s5 = torch.mean(torch.abs(Xi - Xi1), 1)
    s6 = torch.mean((Xi - Xi1)**2 , 1) / n0 
    s7 = torch.mean((Xi - Xi1)**3 , 1) / n0
    s8 = torch.mean((Xi - Xi1)**4 , 1) / n0 ** 2
    
    s9 = torch.mean((Xi - Xi1)**2 * Xi, 1) / n0 
    s10 = torch.mean((Xi - Xi1)**2 * Xi ** 2, 1)/ n0 ** 2
    
    Xi = Xi + torch.reshape(s0, (L0, 1))
    Xi1 = Xi1 + torch.reshape(s1, (L0, 1))
    
    s11 = torch.mean(Xi * Xi1, 1) - s0 * s1
    s11 = s11 / ( torch.mean(Xi1 ** 2, 1) - s1 ** 2 )
    
    s12 = s0 * torch.mean(Xi1 ** 2,1) - s1 * torch.mean(Xi * Xi1, 1)
    s12 = s12 / (torch.mean(Xi1 ** 2, 1) - s1 ** 2)
    
    # Jump intensity
    tmp = abs(Xi - Xi1)
    
    thres = [1e-5 * 1, 1e-5 * 5, 1e-4 * 1, 1e-4 * 5, 1e-3 * 5, 1e-3 * 1,
           1e-2 * 1, 1e-2 * 5, 1e-1 * 1, 1e-1 * 5,
             1.0, 1.5, 2, 2.5, 3]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)

    j_int = torch.column_stack(thres_tmp)
    
    # Jump magnitude
    tmp = Xi - Xi1
    num = 15
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, 
                               s10, s11, s12, j_int, mag_q)) ) 


def SQRJ_summary(X):
    """
    X: torch size: [L,n]
    """
    
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    # Pseudo-likelihood
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    
    s2 = torch.mean((X0 - s0) * (X1 - s1),1,keepdim=True)/ n0 ** (1/2)
    s3 = torch.mean((X0 - s0)**2,1, keepdim=True)/ n0 ** (1/2)
    s4 = torch.mean((X1 - s1)**2,1, keepdim=True)/ n0 ** (1/2)
    
    s5 = torch.mean(1/ X0, 1, keepdim=True)
    s6 = torch.mean(X1/ X0, 1, keepdim=True)
    s7 = torch.mean(X1 ** 2/ X0, 1, keepdim=True)/ n0 ** (1/2)
    s8 = torch.log(torch.max(X0, 1e-10*torch.ones(1)))
    s8 = torch.mean(s8, 1, keepdim=True)
    
    # GMM
    s9 = torch.mean((X0 - s0)**2 * (X1 - s1), 1, keepdim=True)/ n0 
    s10 = torch.mean((X0 - s0) * (X1 - s1)**2, 1, keepdim=True)/ n0 
    s11 = torch.mean((X0 - s0)**2 * (X1 - s1)**2, 1, keepdim=True)/ n0 ** (3/2)
    
    
    # Jump intensity
    tmp = abs(X1 - X0)
    thres = [1e-5 * 3, 1e-5 * 6, 1e-5 * 9, 1e-4 * 3, 1e-4 * 6, 1e-4 * 9, 1e-3 * 3, 1e-3 * 6, 1e-3 * 9,
           1e-2 * 3, 1e-2 * 6, 1e-2 * 9, 1e-1 * 3, 1e-1 * 6, 1e-1 * 9,
             1.0, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)

    j_int = torch.column_stack(thres_tmp)
    
    # Jump magnitude
    tmp = X1 - X0
    num = 33
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, 
                               s10, s11, j_int, mag_q)) )

def PBJD_summary(X, delta):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    Xi = X[:,range(1,n0)]
    Xi1 = X[:,range(0,n0-1)]
    
    # mean
    s0 = torch.sum((Xi - Xi1), 1) / (delta* (n0-1))
    s1 = torch.mean(torch.abs(Xi - Xi1), 1) / n0
    s2 = torch.mean((Xi - Xi1)**2 , 1) / n0
    s3 = torch.mean((Xi - Xi1)**3 , 1) / n0 
    s4 = torch.mean((Xi - Xi1)**4 , 1) / n0 ** 2
    
    tmp = (Xi - Xi1 - torch.reshape(s0, (L0, 1)) * delta) ** 2/delta
    
    # sigma
    s5 = torch.mean(tmp, 1)/n0
    
    # Jump intensity
    tmp = (Xi - Xi1)
    
    thres = [1e-5 * 3, 1e-5 * 6, 1e-5 * 9, 1e-4 * 3, 1e-4 * 6, 1e-4 * 9, 1e-3 * 3, 1e-3 * 6, 1e-3 * 9,
           1e-2 * 3, 1e-2 * 6, 1e-2 * 9, 1e-1 * 3, 1e-1 * 6, 1e-1 * 9,
             1.0, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)
    
    j_int1 = torch.column_stack(thres_tmp)
    
    thres_tmp2 = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp < -thres[i] ), 1) /n0
        thres_tmp2.append(temp)
    
    j_int2 = torch.column_stack(thres_tmp2)
    
    # Jump magnitude
    tmp = Xi - Xi1
    num = 33
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, 
                               j_int1, j_int2, mag_q)) ) 

def PBJD_RDA_summary(X, delta):
    """
    X: torch size: [L,n]
    """
    L0 = X.size()[0]
    n0 = X.size()[1]
    
    Xi = X[:,range(1,n0)]
    Xi1 = X[:,range(0,n0-1)]
    
    # mean
    s0 = torch.sum((Xi - Xi1), 1) / (delta* (n0-1))
    s1 = torch.mean(torch.abs(Xi - Xi1), 1) / n0
    s2 = torch.mean((Xi - Xi1)**2 , 1) / n0
    s3 = torch.mean((Xi - Xi1)**3 , 1) / n0 
    s4 = torch.mean((Xi - Xi1)**4 , 1) / n0 ** 2
    
    tmp = (Xi - Xi1 - torch.reshape(s0, (L0, 1)) * delta) ** 2/delta
    
    # sigma
    s5 = torch.mean(tmp, 1)/n0
    
    # Jump intensity
    tmp = (Xi - Xi1)
    
    thres = [1e-7 * 3, 1e-7 * 6, 1e-7 * 9, 
             1e-6 * 3, 1e-6 * 6, 1e-6 * 9, 
             1e-5 * 3, 1e-5 * 6, 1e-5 * 9, 
             1e-4 * 3, 1e-4 * 6, 1e-4 * 9, 
             1e-3 * 3, 1e-3 * 6, 1e-3 * 9,
             1e-2 * 3, 1e-2 * 6, 1e-2 * 9]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)
    
    j_int1 = torch.column_stack(thres_tmp)
    
    thres_tmp2 = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp < -thres[i] ), 1) /n0
        thres_tmp2.append(temp)
    
    j_int2 = torch.column_stack(thres_tmp2)
    
    # Jump magnitude
    tmp = Xi - Xi1
    num = 33
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, 
                               j_int1, j_int2, mag_q)) ) 


def BOUJ_summary(sim_data):
    """
    sim_data: torch size: [2, L,n]
    """
    L0 = sim_data.size()[1]
    n0 = sim_data.size()[2]
    
    X = sim_data[0]
    Y = sim_data[1]
    
    X0 = X[:,:-1]
    X1 = X[:,1:]
    
    Y0 = Y[:,:-1]
    Y1 = Y[:,1:]
    
    # Pseudo
    s0 = torch.mean(X0, 1, keepdim=True) # mean of x_{i-1}
    s1 = torch.mean(X1, 1, keepdim=True) # mean of x_i
    s2 = torch.mean((X0 - s0) * (X1 - s1), 1, keepdim=True) /n0 ** (1/2)
    s3 = torch.mean((X0 - s0)**2, 1, keepdim=True) / n0 ** (1/2)
    s4 = torch.mean((X1 - s1)**2, 1, keepdim=True) / n0 ** (1/2)
    
    s5 = torch.mean(Y0, 1, keepdim=True) # mean of x_{i-1}
    s6 = torch.mean(Y1, 1, keepdim=True) # mean of x_i
    s7 = torch.mean((Y0 - s5) * (Y1 - s6), 1, keepdim=True)/n0 ** (1/2)
    s8 = torch.mean((Y0 - s5)**2, 1, keepdim=True) / n0 ** (1/2)
    s9 = torch.mean((Y1 - s6)**2, 1, keepdim=True) / n0 ** (1/2)
    
    s10 = torch.mean((X0 - s0) * (Y0 - s5), 1 ,keepdim = True) / n0 ** (1/2)
    s11 = torch.mean((X1 - s1) * (Y0 - s5), 1 ,keepdim = True) / n0 ** (1/2)
    s12 = torch.mean((X0 - s1) * (Y1 - s6), 1 ,keepdim = True) / n0 ** (1/2)
    s13 = torch.mean((X1 - s1) * (Y1 - s6), 1 ,keepdim = True) / n0 ** (1/2)
    
    # GMM
    s14 = torch.mean(torch.abs(X1 - X0), 1)
    s15 = torch.mean((X1 - X0)**2 , 1) / n0 
    s16 = torch.mean((X1 - X0)**3 , 1) / n0
    s17 = torch.mean((X1 - X0)**4 , 1) / n0 ** 2
    
    s18 = torch.mean((X1 - X0)**2 * X0, 1) / n0 
    s19 = torch.mean((X1 - X0)**2 * X1 ** 2, 1)/ n0 ** 2
    
    s20 = torch.mean(torch.abs(Y1 - Y0), 1)
    s21 = torch.mean((Y1 - Y0)**2 , 1) / n0 
    s22 = torch.mean((Y1 - Y0)**3 , 1) / n0
    s23 = torch.mean((Y1 - Y0)**4 , 1) / n0 ** 2
    
    s24 = torch.mean((Y1 - Y0)**2 * Y0, 1) / n0 
    s25 = torch.mean((Y1 - Y0)**2 * Y1 ** 2, 1)/ n0 ** 2
    
    s26 = torch.mean(X1 - Y1, 1 ,keepdim = True) / n0 ** (1/2)
    s27 = torch.mean((X1 - Y1)**2, 1 ,keepdim = True) / n0 
    s28 = torch.mean((X1 - Y1)**3, 1 ,keepdim = True) / n0 ** (3/2)
    s29 = torch.mean((X1 - Y1)**4, 1 ,keepdim = True) / n0 ** 2
    
    
    # Jump intensity
    tmp = torch.abs(X1 - X0)
    thres = [1e-5 * 3, 1e-5 * 6, 1e-5 * 9, 1e-4 * 3, 1e-4 * 6, 1e-4 * 9, 1e-3 * 3, 1e-3 * 6, 1e-3 * 9,
           1e-2 * 3, 1e-2 * 6, 1e-2 * 9, 1e-1 * 3, 1e-1 * 6, 1e-1 * 9,
             1.0, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25]
    thres_tmp = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp.append(temp)
    j_int = torch.column_stack(thres_tmp)
    
    # Jump intensity
    tmp = torch.abs(Y1 - Y0)
    thres_tmp2 = []
    for i in range(len(thres)):
        temp = torch.sum( (tmp > thres[i] ), 1) /n0
        thres_tmp2.append(temp)
    j_int2 = torch.column_stack(thres_tmp2)
    
    
    # Jump magnitude
    tmp = torch.abs(X1 - X0)
    num = 24
    q = []
    for i in range(num+1):
         q.append(i/num)
    
    q = torch.tensor(q)
    mag_q = torch.transpose(torch.quantile(tmp, q, 1), 0, 1)
    
    # Jump magnitude
    tmp2 = torch.abs(Y1 - Y0)
    num = 24
    q2 = []
    for i in range(num+1):
         q2.append(i/num)
    
    q2 = torch.tensor(q2)
    mag_q2 = torch.transpose(torch.quantile(tmp2, q2, 1), 0, 1)
    
    
    return(torch.column_stack((s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, 
                              s10, s11, s12, s13, s14, s15, s16, s17, s18, s19,
                              s20, s21, s22, s23, s24, s25, s26, s27, s28, s29,
                               j_int, j_int2, mag_q, mag_q2)) ) 

In [ ]:
def weighted_quantile(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

def mad_np(sam):
    return np.median(np.absolute(sam - np.mean(sam)))

In [ ]:
import torch.optim as optim
import copy
import torch.nn as nn
import torch.nn.functional as F
from DNN_module import Net
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

def con_mad(X, resid, N_EPOCHS = 250, learning_rate = 1e-5, weight_decay = 1e-5, layer_len = 250):
    # X: torch([L,m]) L: nubmer of synthetic data; m: size of input
    # resid: torch([L,p]) p: the number of parameter of interest
    # x0 : torch([1,m]) 
    
    #output: torch([L,p])
    # size
    if torch.cuda.is_available(): 
        dev = "cuda:0" 
    else: 
        dev = "cpu"
    device = torch.device(dev) 
    torch.set_default_device(device)
    X = X.to(device)
    resid = resid.to(device)
    torch.set_default_device('cuda')
    
    resid = torch.max(torch.abs(resid), torch.ones(1) * 1e-30)
    
    Y = torch.log(resid)
    L = X.size()[0]

    L_train = np.floor(L * 0.7).astype(int)
    L_val = L - L_train

    torch.manual_seed(1000)
    indexes = torch.randperm(L_train + L_val)
    #Batch size
    BATCH_SIZE = 64
    
    # Divide Data
    X_train = X[indexes[0:L_train]]
    Y_train = Y[indexes[0:L_train]]

    X_val = X[indexes[L_train:(L_train + L_val)]]
    Y_val = Y[indexes[L_train:(L_train + L_val)]]

    # Use torch.utils.data to create a DataLoader that will take care of creating batches
    dataset = TensorDataset(X_train, Y_train)
    dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, generator=torch.Generator(device=device))

    # Get the dataset size for printing (it is equal to N_SAMPLES)
    dataset_size = len(dataloader.dataset)

    # Define the input and output dimensions
    D_in, D_out = X.size()[1], Y.size()[1]
    
    # Create an instance of the Net class with specified dimensions
    H, H2, H3 = layer_len, layer_len, layer_len
    net_var = Net(D_in = D_in, D_out = D_out, H = H, H2 = H2, H3 = H3)
    best_model_state = copy.deepcopy(net_var.state_dict())
    # Model name
    #model_save_name = 'cond_var.pt'
    #path = F"./{model_save_name}"

    def weighted_mse_loss(input, target, weight):
        return (weight * (input - target) ** 2).sum()
    out_range = [torch.min(Y_train,0).values.detach().cpu().numpy(), torch.max(Y_train,0).values.detach().cpu().numpy()]
    weight_1 = torch.tensor(1/(out_range[1] - out_range[0])**2)
    
    # The nn package also contains definitions of popular loss functions; in this case we will use Mean Squared Error (MSE) as our loss function.
    #loss_fn = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(net_var.parameters(), lr=learning_rate, weight_decay=weight_decay)

    train_error_plt = []
    val_error_plt = []
    
    
    # Loop over epochs
    for epoch in range(N_EPOCHS):
        for id_batch, (x_batch, y_batch) in enumerate(dataloader):
            y_batch_pred = net_var(x_batch)
            #loss = loss_fn(y_batch_pred, y_batch)
            loss = weighted_mse_loss(y_batch, y_batch_pred, weight_1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 50 ==0 and id_batch % 100 == 0:
                loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                print(f"train_loss: {loss/BATCH_SIZE:>7f}  [{current:>5d}/{dataset_size:>5d}]")

        with torch.no_grad():
            net_var.eval()
            theta_pred_train = net_var(X_train)
            # train_loss = loss_fn(theta_pred_train,Y_train) / L_train
            train_loss = weighted_mse_loss(Y_train, theta_pred_train, weight_1) / L_train
            train_error_plt.append(train_loss.to("cpu"))

            theta_pred_val = net_var(X_val)
            #val_loss = loss_fn(Y_val, theta_pred_val) / L_val
            val_loss = weighted_mse_loss(Y_val, theta_pred_val, weight_1) / L_val
            val_error_plt.append(val_loss.to("cpu"))

        if epoch % 30 ==0:
            print(f"Epoch {epoch + 1}\n-------------------------------")
            print(f"train_loss {train_loss:>7f} val_loss {val_loss:>7f}")

        ## Choose Best Model
        if val_error_plt[epoch] == np.min(val_error_plt):
            best=epoch
            best_model_state = copy.deepcopy(net_var.state_dict())

        if epoch % 100 == 49:
            #net_var.load_state_dict(torch.load(path))
            net_var.load_state_dict(best_model_state)
            learning_rate = max(learning_rate * 1e-1, 1e-9)
            
        if val_error_plt[epoch] == math.inf:
            if train_error_plt[epoch] == np.min(train_error_plt):
                best_model_state = copy.deepcopy(net_var.state_dict())
                
    # load net
    net_var = Net(D_in, D_out, H = H, H2 = H2, H3 = H3)
    #net_var.load_state_dict(torch.load(path))
    net_var.load_state_dict(best_model_state)
    net_var.eval()
    print(np.min(val_error_plt))

    return net_var, device, np.min(val_error_plt)

In [1]:
def conf_inf_sd(x0, X, Y, net, tol, h=None, seed = 1001, N_EPOCHS = 250, l_rate = 1e-5, w_rate = 1e-5, layer_len = 250):
    # x0: torch([m])    m: number of summary statistics
    # X: torch([L,m]) L: nubmer of synthetic data (should be cpu)
    # Y: torch([L,p]) L: nubmer of synthetic data (should be cpu)
    # net: neural net in this case
    # tol: \in (0,1)
    # h: numeric >1
    
    if torch.cuda.is_available(): 
        dev = "cuda:0" 
    else: 
        dev = "cpu"
    
    X = X.to("cpu")
    Y = Y.to("cpu")
    net = net.to("cpu")
    x0 = x0.to("cpu")
    

    # size
    L, m = X.size()
    p = Y.size()[1]
        
    # x0 reshaping
    x0 = torch.reshape(x0, (1,m))
    net.eval()
    thetahat = net(x0)
    #resid = Y - net(X)
    
    # Normalizing for stability
    
    c_tmp = torch.quantile(X, torch.tensor([.001,.999], device = "cpu"), 0)
    a = torch.reshape(c_tmp[0], (1, c_tmp.size()[1]))
    b = torch.reshape(c_tmp[1], (1, c_tmp.size()[1]))
    
    X_scale = torch.clone((X - a) / (b - a))
    x0 = torch.clone((x0 - a) / (b - a))
    
    # Distance
    dist = torch.sum((X_scale - x0)**2, 1)
    dist = torch.sqrt(dist)
    sort_dist, indicies = torch.sort(dist)
    
    # new index
    nacc = np.floor(L * tol).astype(int)
    ds = sort_dist[nacc-1]
    wt1 = (dist <= ds)
    
    # weights
    if h is None:
        h = 1/ np.log(torch.sum(wt1))
    
    weights = torch.exp(-dist[wt1]/ds/h)
    X_tmp = X_scale[wt1,:]
    Y_tmp = Y[wt1,:]
    
    X_tmp = X_tmp.to("cpu")
    net.eval()
    resid_tmp = Y_tmp - net(X[wt1,:])
    resid_tmp = resid_tmp.detach()
    
    resid_tmp = resid_tmp.to("cpu")
    
    torch.manual_seed(seed)
    indexes = torch.randperm(X_tmp.size()[0])
    indexes = indexes.to("cpu")
    
    L_var = np.floor(X_tmp.size()[0] * 0.5).astype(int)
    L_adj = X_tmp.size()[0] - L_var
    
    # Divide Data
    X_tmp_train = X_tmp[indexes[0:L_var]]
    resid_tmp_train = resid_tmp[indexes[0:L_var]]
    
    X_tmp_val = X_tmp[indexes[L_var:(L_adj + L_var)]]
    resid_tmp_val = resid_tmp[indexes[L_var:(L_adj + L_var)]]
    weights_val = weights[indexes[L_var:(L_adj + L_var)]]
    
    net_cond, dev, _ = con_mad(X_tmp_train, resid_tmp_train, N_EPOCHS = N_EPOCHS, layer_len = layer_len)
    net_cond.eval()
    sd_x0 = torch.exp(net_cond(x0.to(dev)))
    sd_X = torch.exp(net_cond(X_tmp_val.to(dev)))
    sd_x0 = sd_x0.to("cpu")
    sd_X = sd_X.to("cpu")
    
    adj = thetahat + resid_tmp_val * sd_x0/sd_X
    
    return weights_val, adj

In [2]:
def conf_inf_sd2(x0, X, Y, net, tol, h=None, ln_num = None, 
                 N_EPOCHS = 100, l_rate = None, w_rate = None, seed = None):
    # x0: torch([m])    m: number of summary statistics
    # X: torch([L,m]) L: nubmer of synthetic data (should be cpu objects)
    # Y: torch([L,p]) L: nubmer of synthetic data (should be cpu objects)
    # net: neural net in this case
    # tol: \in (0,1)
    # h: numeric >1
    if torch.cuda.is_available(): 
        dev = "cuda:0" 
    else: 
        dev = "cpu"
    
    X = X.to("cpu")
    Y = Y.to("cpu")
    net = net.to("cpu")
    x0 = x0.to("cpu")
    

    # size
    L, m = X.size()
    p = Y.size()[1]
        
    # x0 reshaping
    x0 = torch.reshape(x0, (1,m))
    net.eval()
    thetahat = net(x0)
    #resid = Y - net(X)
    
    # Normalizing for stability
    
    c_tmp = torch.quantile(X, torch.tensor([.001,.999], device = "cpu"), 0)
    a = torch.reshape(c_tmp[0], (1, c_tmp.size()[1]))
    b = torch.reshape(c_tmp[1], (1, c_tmp.size()[1]))
    
    X_scale = torch.clone((X - a) / (b - a))
    x0 = torch.clone((x0 - a) / (b - a))
    
    # Distance
    dist = torch.sum((X_scale - x0)**2, 1)
    dist = torch.sqrt(dist)
    sort_dist, indicies = torch.sort(dist)
    
    # new index
    nacc = np.floor(L * tol).astype(int)
    ds = sort_dist[nacc-1]
    wt1 = (dist <= ds)
    
    # weights
    if h is None:
        h = 1/ np.log(torch.sum(wt1))
        
    if ln_num is None:
        ln_num = 10
    
    if l_rate is None:
        l_rate = 1e-5
    
    if w_rate is None:
        w_rate = 1e-5
    
    if seed is None:
        seed = 1001
    
    weights = torch.exp(-dist[wt1]/ds/h)
    X_tmp = X_scale[wt1,:]
    Y_tmp = Y[wt1,:]
    
    X_tmp = X_tmp.to("cpu")
    resid_tmp = Y_tmp - net(X[wt1,:])
    resid_tmp = resid_tmp.detach()
    
    resid_tmp = resid_tmp.to("cpu")
    
    
    l_rate_list = [l_rate, l_rate * 1e-1, l_rate * 10]
    w_rate_list = [w_rate, w_rate * 1e-1, w_rate * 10]
    random.seed(10)
    l_rate_list = random.choices(l_rate_list, k=ln_num)
    w_rate_list = random.choices(w_rate_list, k=ln_num)
    
    # Create an instance of the Net class with specified dimensions
    H, H2, H3 = 256, 256, 256
    D_in, D_out = X_tmp.size()[1], resid_tmp.size()[1]
    net_cond_best = Net(D_in = D_in, D_out = D_out, H = H, H2 = H2, H3 = H3)

    val_min_0 = float('inf')
    for k in range(ln_num):
        torch.manual_seed(seed+ln_num)
        indexes = torch.randperm(X_tmp.size()[0])
        indexes = indexes.to("cpu")
    
        L_var = np.floor(X_tmp.size()[0] * 0.5).astype(int)
        L_adj = X_tmp.size()[0] - L_var
    
        # Divide Data
        X_tmp_train = X_tmp[indexes[0:L_var]]
        resid_tmp_train = resid_tmp[indexes[0:L_var]]
    
        X_tmp_val = X_tmp[indexes[L_var:(L_adj + L_var)]]
        resid_tmp_val = resid_tmp[indexes[L_var:(L_adj + L_var)]]
        weights_val = weights[indexes[L_var:(L_adj + L_var)]]
    
        l_rate = l_rate_list[k]
        w_rate = w_rate_list[k]
        
        net_cond, dev, val_min = con_mad(X_tmp_train, resid_tmp_train, N_EPOCHS = N_EPOCHS, 
                                         learning_rate = l_rate, weight_decay = w_rate)
        if val_min <= val_min_0:
            best_model_state = copy.deepcopy(net_cond.state_dict())
            val_min_0 = val_min
        print("iter: ", k, "Val= ", val_min_0)
        
    net_cond_best.load_state_dict(best_model_state)
    net_cond_best = net_cond_best.to("cpu")
    net_cond_best.eval()
    sd_x0 = torch.exp(net_cond_best(x0.to("cpu")))
    sd_X = torch.exp(net_cond_best(X_tmp_val.to("cpu")))
    sd_x0 = sd_x0.to("cpu")
    sd_X = sd_X.to("cpu")
    
    adj = thetahat + resid_tmp_val * sd_x0/sd_X
    
    return weights_val, adj

In [ ]:
from __future__ import division
import numpy as np
import scipy.stats.kde as kde

def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode

    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results

    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
         x_hpd = x[(x > value[0]) & (x < value[1])]
         y_hpd = y[(x > value[0]) & (x < value[1])]
         modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes